In [52]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

In [ ]:
##ADDED
file_name = "NF-CSE-CIC-IDS2018-v2.parquet"
test_data = pd.read_parquet(file_name)
##

In [54]:
## ADDED
test_data.Label.value_counts()
##

Label
0    12732
1     6989
Name: count, dtype: int64

In [55]:
##ADDED
test_data.rename(columns=lambda x: x.strip(), inplace=True)
test_data['IPV4_SRC_ADDR'] = test_data["IPV4_SRC_ADDR"].apply(str)
test_data['L4_SRC_PORT'] = test_data["L4_SRC_PORT"].apply(str)
test_data['IPV4_DST_ADDR'] = test_data["IPV4_DST_ADDR"].apply(str)
test_data['L4_DST_PORT'] = test_data["L4_DST_PORT"].apply(str)
##

KeyError: 'L4_SRC_PORT'

In [ ]:
# ADDED
test_data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)
##

In [ ]:
##ADDED
test_data.Attack.unique()
##

array(['DoS attacks-GoldenEye', 'Benign', 'FTP-BruteForce',
       'DoS attacks-Hulk', 'Infilteration', 'DoS attacks-SlowHTTPTest',
       'DoS attacks-Slowloris', 'SQL Injection', 'SSH-Bruteforce',
       'Brute Force -Web', 'Brute Force -XSS'], dtype=object)

In [ ]:
## ADDED
test_data = test_data.groupby(by='Attack').sample(frac=0.02, random_state=13)
##

In [ ]:
##ADDED
test_data.groupby(by="Attack").count()
##

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,12732,12732,12732,12732,12732,12732,12732,12732,12732,12732,...,12732,12732,12732,12732,12732,12732,12732,12732,12732,12732
Brute Force -Web,588,588,588,588,588,588,588,588,588,588,...,588,588,588,588,588,588,588,588,588,588
Brute Force -XSS,336,336,336,336,336,336,336,336,336,336,...,336,336,336,336,336,336,336,336,336,336
DoS attacks-GoldenEye,1957,1957,1957,1957,1957,1957,1957,1957,1957,1957,...,1957,1957,1957,1957,1957,1957,1957,1957,1957,1957
DoS attacks-Hulk,1149,1149,1149,1149,1149,1149,1149,1149,1149,1149,...,1149,1149,1149,1149,1149,1149,1149,1149,1149,1149
DoS attacks-SlowHTTPTest,678,678,678,678,678,678,678,678,678,678,...,678,678,678,678,678,678,678,678,678,678
DoS attacks-Slowloris,1031,1031,1031,1031,1031,1031,1031,1031,1031,1031,...,1031,1031,1031,1031,1031,1031,1031,1031,1031,1031
FTP-BruteForce,144,144,144,144,144,144,144,144,144,144,...,144,144,144,144,144,144,144,144,144,144
Infilteration,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,...,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000


In [ ]:
# X = data.drop(columns=["Attack", "Label", "FLOW_START_MILLISECONDS", "FLOW_END_MILLISECONDS",
#                        "SRC_TO_DST_IAT_MIN", "SRC_TO_DST_IAT_MAX", "SRC_TO_DST_IAT_AVG",
#                        "SRC_TO_DST_IAT_STDDEV", "DST_TO_SRC_IAT_MIN", "DST_TO_SRC_IAT_MAX",
#                        "DST_TO_SRC_IAT_AVG", "DST_TO_SRC_IAT_STDDEV"])
# y = data[["Attack", "Label"]]

# X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=0.3, random_state=13, stratify=y)
##Added
X = test_data.drop(columns=["Attack", "Label"])
y = test_data[["Attack", "Label"]]

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=13, stratify=y)
##

In [ ]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [ ]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [ ]:
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [ ]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h
792540,145.82.199.44,190.201.33.51,4.759143e-08,9.295172e-08,0.000026,4.028733e-07,0.000016,2.685822e-07,1.342911e-07,1.341848e-07,...,0.0,0.000138,0.000000,4.759143e-08,4.759143e-08,4.759143e-08,4.759143e-08,-1.342911e-07,4.944168e-08,"[4.7591425271285124e-08, 9.295171612226107e-08..."
822547,145.82.199.44,190.201.33.51,7.517051e-08,7.127150e-08,0.000051,6.363371e-07,0.000000,0.000000e+00,2.121124e-07,2.119444e-07,...,0.0,0.013626,0.000000,7.517051e-08,7.517051e-08,7.517051e-08,7.517051e-08,-2.121124e-07,7.809297e-08,"[7.517050629062862e-08, 7.127149899596397e-08,..."
515011,145.82.199.44,190.201.33.51,1.543268e-07,1.463221e-07,0.000426,2.177358e-06,0.000626,8.709434e-07,6.240245e-08,6.389454e-08,...,0.0,0.000250,0.000218,1.543268e-07,1.543268e-07,1.543268e-07,1.543268e-07,-4.354717e-07,1.603267e-07,"[1.5432682632103823e-07, 1.4632207217901126e-0..."
734682,145.82.199.44,182.129.67.88,2.430984e-08,4.748001e-08,0.009488,6.242262e-06,0.000000,0.000000e+00,6.800421e-08,6.800421e-08,...,0.0,0.000000,0.000000,2.430984e-08,2.430984e-08,2.430984e-08,2.430984e-08,-6.859629e-08,2.525495e-08,"[2.4309839676189835e-08, 4.7480009343663414e-0..."
6691,145.82.199.44,190.201.33.51,3.105224e-09,2.944160e-09,0.000014,2.628650e-08,0.000000,0.000000e+00,1.255605e-09,1.285628e-09,...,0.0,0.000004,0.000000,3.105224e-09,3.105224e-09,3.105224e-09,3.105224e-09,-8.762166e-09,3.225949e-09,"[3.105224221484079e-09, 2.944159829496005e-09,..."


In [ ]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [ ]:
# Training graph
train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
           ["h", "Label", "Attack"], create_using=nx.MultiGraph())
train_g = train_g.to_directed()
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack"], create_using=nx.MultiGraph())

test_g = test_g.to_directed()
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [ ]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [ ]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [ ]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [ ]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi = dgi.to('cuda')

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [ ]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [ ]:
# Convert to GPU
train_g = train_g.to('cuda')

In [ ]:
dgi.load_state_dict(torch.load('best_dgi_CSE_self_collected.pkl'))

/tmp/user/1000/ipykernel_58786/204607001.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dgi.load_state_dict(torch.load('best_dgi_CSE_v3.pkl'))


<All keys matched successfully>

In [ ]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [ ]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [ ]:
# Convert to GPU
test_g = test_g.to('cuda')

In [ ]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [ ]:
df_train = pd.DataFrame(training_emb, )
df_train["Attack"] = lab_enc.inverse_transform(
        train_g.edata['Attack'].detach().cpu().numpy())
df_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb, )
df_test["Attack"] = lab_enc.inverse_transform(
        test_g.edata['Attack'].detach().cpu().numpy())
df_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

In [ ]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,248,249,250,251,252,253,254,255,Attack,Label
0,0.115612,0.110404,0.058813,-0.116445,-0.244497,0.288873,0.406665,0.073494,0.114636,-0.097980,...,-0.083656,0.104936,0.417963,-0.136752,0.102994,-0.253011,-0.385048,-0.103878,Infilteration,1
1,0.115612,0.110404,0.058813,-0.116445,-0.244497,0.288873,0.406665,0.073494,0.114636,-0.097980,...,-0.083656,0.104936,0.417963,-0.136752,0.102994,-0.253011,-0.385048,-0.103878,DoS attacks-SlowHTTPTest,1
2,0.115612,0.110404,0.058813,-0.116445,-0.244497,0.288873,0.406665,0.073494,0.114636,-0.097980,...,-0.083656,0.104936,0.417963,-0.136752,0.102994,-0.253011,-0.385048,-0.103878,Benign,0
3,0.115612,0.110404,0.058813,-0.116445,-0.244497,0.288873,0.406665,0.073494,0.114636,-0.097980,...,-0.083656,0.104936,0.417963,-0.136752,0.102994,-0.253011,-0.385048,-0.103878,DoS attacks-GoldenEye,1
4,0.115612,0.110404,0.058813,-0.116445,-0.244497,0.288873,0.406665,0.073494,0.114636,-0.097980,...,-0.083656,0.104936,0.417963,-0.136752,0.102994,-0.253011,-0.385048,-0.103878,Infilteration,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27603,0.100981,0.081071,0.045001,-0.116135,-0.157482,0.214536,0.341279,0.067098,0.080982,-0.069247,...,-0.085523,0.087880,0.336211,-0.131436,0.105387,-0.188004,-0.346915,-0.080704,Brute Force -XSS,1
27604,0.089549,0.077124,0.054228,-0.108793,-0.159717,0.219976,0.345229,0.068070,0.088760,-0.075872,...,-0.081815,0.084443,0.335564,-0.127771,0.096480,-0.182200,-0.341740,-0.076445,Brute Force -XSS,1
27605,0.089549,0.077124,0.054228,-0.108793,-0.159717,0.219976,0.345229,0.068070,0.088760,-0.075872,...,-0.081815,0.084443,0.335564,-0.127771,0.096480,-0.182200,-0.341740,-0.076445,Brute Force -XSS,1
27606,0.089494,0.077101,0.054244,-0.108721,-0.159782,0.219996,0.345114,0.068109,0.088761,-0.075871,...,-0.081716,0.084476,0.335499,-0.127687,0.096431,-0.182149,-0.341629,-0.076384,Brute Force -XSS,1


# Embeddings CBLOF  Embeddings

In [ ]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
import gc

from tqdm import tqdm
import itertools

In [ ]:
benign_train_samples = df_train[df_train.Label == 0].drop(columns=["Label", "Attack"])
normal_train_samples = df_train.drop(columns=["Label", "Attack"])

train_labels = df_train["Label"]
test_labels = df_test["Label"]

test_samples = df_test.drop(columns=["Label", "Attack"])

In [ ]:
test_samples

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,0.115246,0.110145,0.059667,-0.116374,-0.242968,0.288304,0.405851,0.073335,0.114405,-0.098377,...,-0.253175,0.074608,-0.083086,0.104216,0.416556,-0.136001,0.102983,-0.252348,-0.384348,-0.104107
1,0.115246,0.110145,0.059667,-0.116374,-0.242968,0.288304,0.405851,0.073335,0.114405,-0.098377,...,-0.253175,0.074608,-0.083086,0.104216,0.416556,-0.136001,0.102983,-0.252348,-0.384348,-0.104107
2,0.115246,0.110145,0.059667,-0.116374,-0.242968,0.288304,0.405851,0.073335,0.114405,-0.098377,...,-0.253175,0.074608,-0.083086,0.104216,0.416556,-0.136001,0.102983,-0.252348,-0.384348,-0.104107
3,0.115246,0.110145,0.059667,-0.116374,-0.242968,0.288304,0.405851,0.073335,0.114405,-0.098377,...,-0.253175,0.074608,-0.083086,0.104216,0.416556,-0.136001,0.102983,-0.252348,-0.384348,-0.104107
4,0.115246,0.110145,0.059667,-0.116374,-0.242968,0.288304,0.405851,0.073335,0.114405,-0.098377,...,-0.253175,0.074608,-0.083086,0.104216,0.416556,-0.136001,0.102983,-0.252348,-0.384348,-0.104107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11829,0.090627,0.077372,0.054074,-0.109997,-0.158362,0.219675,0.346844,0.067430,0.088658,-0.075805,...,-0.223379,0.050242,-0.083318,0.084135,0.336461,-0.129031,0.097250,-0.182881,-0.343743,-0.077370
11830,0.090627,0.077372,0.054074,-0.109997,-0.158362,0.219675,0.346844,0.067430,0.088658,-0.075805,...,-0.223379,0.050242,-0.083318,0.084135,0.336461,-0.129031,0.097250,-0.182881,-0.343743,-0.077370
11831,0.090603,0.077371,0.053742,-0.109934,-0.158672,0.219607,0.346842,0.067489,0.088660,-0.075662,...,-0.223494,0.050372,-0.083484,0.084252,0.336668,-0.129210,0.097298,-0.182937,-0.343660,-0.077258
11832,0.089715,0.077129,0.054095,-0.108870,-0.160221,0.220254,0.345400,0.068101,0.088936,-0.075851,...,-0.223314,0.051533,-0.081978,0.084836,0.336137,-0.127901,0.096491,-0.182456,-0.342136,-0.076427


In [ ]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [ ]:
from pyod.models.cblof import CBLOF
n_est = [2,3,5,7,9,10]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    try:
        clf_if.fit(benign_train_samples)
    except Exception as e:
        continue
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

 64%|██████▍   | 23/36 [00:01<00:00, 14.47it/s]/home/kienho/miniforge3/envs/nids-env/lib/python3.9/site-packages/sklearn/base.py:1473: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/kienho/miniforge3/envs/nids-env/lib/python3.9/site-packages/pyod/models/cblof.py:187: UserWarning: The chosen clustering for CBLOF forms 8 clusterswhich is inconsistent with n_clusters (9).
  warnings.warn("The chosen clustering for CBLOF forms {0} clusters"
 69%|██████▉   | 25/36 [00:01<00:00, 12.50it/s]/home/kienho/miniforge3/envs/nids-env/lib/python3.9/site-packages/sklearn/base.py:1473: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/kienho/miniforge3/envs/nids-env/lib/python3.9/site-packages/pyod/models/cblof.py:187: UserWarning: The chosen c

NameError: name 'best_params' is not defined

In [ ]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    try:
        clf_if.fit(normal_train_samples)
    except Exception as e:
        continue
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [02:45<00:00,  4.59s/it]

{'n_estimators': 5, 'con': 0.001}
0.4802652865523067
              precision    recall  f1-score   support

           0     0.6380    0.4851    0.5511    190984
           1     0.3472    0.4988    0.4094    104856

    accuracy                         0.4899    295840
   macro avg     0.4926    0.4919    0.4803    295840
weighted avg     0.5349    0.4899    0.5009    295840



In [ ]:
# HBOS  Embeddings

In [ ]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [ ]:
from pyod.models.hbos import HBOS

n_est = [5,10,15,20,25,30]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [03:32<00:00,  5.90s/it]

{'n_estimators': 10, 'con': 0.001}
0.4828888816018324
              precision    recall  f1-score   support

           0     0.6333    0.6266    0.6299    190984
           1     0.3327    0.3391    0.3359    104856

    accuracy                         0.5247    295840
   macro avg     0.4830    0.4828    0.4829    295840
weighted avg     0.5267    0.5247    0.5257    295840



In [ ]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [06:46<00:00, 11.29s/it]

{'n_estimators': 10, 'con': 0.001}
0.4802652865523067
              precision    recall  f1-score   support

           0     0.6380    0.4851    0.5511    190984
           1     0.3472    0.4988    0.4094    104856

    accuracy                         0.4899    295840
   macro avg     0.4926    0.4919    0.4803    295840
weighted avg     0.5349    0.4899    0.5009    295840



In [ ]:
##  PCA  Emb

In [ ]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [02:20<00:00,  3.90s/it]

{'n_estimators': 5, 'con': 0.001}
0.26168466867650286
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000    190984
           1     0.3544    1.0000    0.5234    104856

    accuracy                         0.3544    295840
   macro avg     0.1772    0.5000    0.2617    295840
weighted avg     0.1256    0.3544    0.1855    295840




/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capit

In [ ]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [02:52<00:00,  4.80s/it]

{'n_estimators': 5, 'con': 0.001}
0.36201354908552197
              precision    recall  f1-score   support

           0     0.6175    0.1416    0.2303    190984
           1     0.3496    0.8403    0.4937    104856

    accuracy                         0.3892    295840
   macro avg     0.4835    0.4909    0.3620    295840
weighted avg     0.5225    0.3892    0.3237    295840



In [ ]:
##  IF  Emb

In [ ]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

  0%|          | 0/24 [00:00<?, ?it/s]

100%|██████████| 24/24 [00:43<00:00,  1.82s/it]

{'n_estimators': 20, 'con': 0.1}
0.49036920021064834
              precision    recall  f1-score   support

           0     0.6434    0.7425    0.6894    190984
           1     0.3481    0.2504    0.2913    104856

    accuracy                         0.5681    295840
   macro avg     0.4958    0.4965    0.4904    295840
weighted avg     0.5387    0.5681    0.5483    295840



In [ ]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [01:19<00:00,  3.33s/it]

{'n_estimators': 20, 'con': 0.001}
0.3945746014854401
              precision    recall  f1-score   support

           0     0.6460    1.0000    0.7850    190984
           1     0.9910    0.0021    0.0042    104856

    accuracy                         0.6463    295840
   macro avg     0.8185    0.5010    0.3946    295840
weighted avg     0.7683    0.6463    0.5082    295840

